In [159]:
import numpy as np
import pandas as pd

In [173]:
long_lat_census_data = pd.read_csv("data/census_block_loc.csv")
block_census_data = pd.read_csv("data/nyc_census_tracts.csv")
restaurant_data = pd.read_csv("preprocessed_data/RestWaterRodent_data.csv.zip")

long_lat_census_data['BlockCode'] = long_lat_census_data['BlockCode'].astype(str)
block_census_data["BlockCode_join"] = block_census_data['CensusTract'].astype(str)
long_lat_census_data['BlockCode_join'] = long_lat_census_data['BlockCode'].str[:11]

block_census_data = block_census_data.groupby("BlockCode_join", as_index=True)[['TotalPop', 'Hispanic', 'White', "Black", "Native", 'Asian', "IncomePerCap", "Poverty", "Unemployment"]].mean().reset_index()

print(block_census_data.shape)
block_census_data.head()


(2167, 10)


,BlockCode_join,TotalPop,Hispanic,White,Black,Native,Asian,IncomePerCap,Poverty,Unemployment
0,36005000100,7703.0,29.9,6.1,60.9,0.2,1.6,2440.0,NaN,NaN
1,36005000200,5403.0,75.8,2.3,16.0,0.0,4.2,22180.0,20.0,7.7
2,36005000400,5915.0,62.7,3.6,30.7,0.0,0.3,27700.0,13.2,9.5
3,36005001600,5879.0,65.1,1.6,32.4,0.0,0.0,17526.0,26.3,8.7
4,36005001900,2591.0,55.4,9.0,29.0,0.0,2.1,17986.0,37.1,19.2


In [174]:
print(len(long_lat_census_data["BlockCode_join"].unique()))
print(long_lat_census_data.shape)
long_lat_census_data.head()

2995
(38396, 6)


,Latitude,Longitude,BlockCode,County,State,BlockCode_join
0,40.48,-74.280000,340230076002012,Middlesex,NJ,34023007600
1,40.48,-74.276834,340230076005000,Middlesex,NJ,34023007600
2,40.48,-74.273668,340230076003018,Middlesex,NJ,34023007600
3,40.48,-74.270503,340230076003004,Middlesex,NJ,34023007600
4,40.48,-74.267337,340230074021000,Middlesex,NJ,34023007402


In [175]:
merged_block_data = pd.merge(long_lat_census_data , block_census_data).drop_duplicates(subset=["BlockCode_join"])
merged_block_data = merged_block_data.dropna().reset_index()

In [216]:
restaurant_csv_path = "preprocessed_data/RestWaterRodent_data.csv.zip"
df_insp = pd.read_csv(restaurant_csv_path).copy()
df_insp = df_insp.dropna()
df_insp.index = np.arange(0,len(df_insp.index))
df_insp.shape

(44786, 22)

In [177]:
print(merged_block_data.columns)
print("Size of restaurant data: ", df_insp.shape)
print("Size of census data: ", merged_block_data.shape)
# Drop duplicate rest.
df_insp = df_insp.drop_duplicates(subset=["CAMIS"]).reset_index()
print("Size of restaurant data: ", df_insp.shape)




Index(['index', 'Latitude', 'Longitude', 'BlockCode', 'County', 'State',
       'BlockCode_join', 'TotalPop', 'Hispanic', 'White', 'Black', 'Native',
       'Asian', 'IncomePerCap', 'Poverty', 'Unemployment'],
      dtype='object')
Size of restaurant data:  (44786, 22)
Size of census data:  (2104, 16)
Size of restaurant data:  (17839, 23)


In [229]:
df_insp_Lat = df_insp['Latitude']
df_insp_Long = df_insp['Longitude']
c_shape = merged_block_data.shape[0]
df_2020_c_Lat = merged_block_data['Latitude']
df_2020_c_Long = merged_block_data['Longitude']
df_2020_c_pop = merged_block_data['TotalPop']
df_2020_c_hisp = merged_block_data['Hispanic']
df_2020_c_white = merged_block_data['White']
df_2020_c_black = merged_block_data['Black']
df_2020_c_native = merged_block_data['Native']
df_2020_c_asian = merged_block_data['Asian']
df_2020_c_income = merged_block_data['IncomePerCap']
df_2020_c_unemp = merged_block_data['Unemployment']

rest_id = []
ClosestDis = []
ClosestBlockID = []
ClosestPop = []
ClosestWhite = []
ClosestHisp = []
ClosestBlack = []
ClosestNative = []
ClosestAsian = []
ClosestInc = []
ClosestUnemp = []

for i in range(df_insp.shape[0]):
    loc1 = np.array([df_insp_Lat[i], df_insp_Long[i]])
    camis = df_insp.loc[i, "CAMIS"]
    dis = []
    for j in range(c_shape):
        loc2 = ([df_2020_c_Lat[j], df_2020_c_Long[j]])
        tem_dist = np.linalg.norm(loc1-loc2)
        dis.append(tem_dist)
    index = np.argmin(dis)

    rest_id.append(camis)
    ClosestDis.append(dis[index])
    ClosestPop.append(df_2020_c_pop[index])
    ClosestWhite.append(df_2020_c_white[index])
    ClosestHisp.append(df_2020_c_hisp[index])
    ClosestBlack.append(df_2020_c_black[index])
    ClosestNative.append(df_2020_c_native[index])
    ClosestAsian.append(df_2020_c_asian[index])
    ClosestInc.append(df_2020_c_income[index])
    ClosestUnemp.append(df_2020_c_unemp[index])
    if(i % 1000 == 0):
        print(i)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000


In [278]:
df_insp["CAMIS"] = rest_id
df_insp["Pop"] = ClosestPop
df_insp["White"] = ClosestWhite
df_insp["Black"] = ClosestBlack
df_insp["Hispanic"] = ClosestHisp
df_insp["Native"] = ClosestNative
df_insp["Asian"] = ClosestAsian
df_insp["Inc"] = ClosestInc
df_insp["Unemp"] = ClosestUnemp
df_insp.head()

,Unnamed: 0,CAMIS,GRADE DATE,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,Latitude,Longitude,BBL,...,INSPECTION_DATE,RESULT,Pop,White,Black,Hispanic,Native,Asian,Inc,Unemp
0,3,30112340,2019-03-04,2.801389,2.913240,2.792569,3.0,40.662652,-73.962081,3.011970e+09,...,2016-09-19 15:25:51,0.0,2501.0,12.2,63.2,18.4,2.4,1.6,35028.0,11.5
1,4,30112340,2021-08-05,2.801389,2.913240,2.634818,2.0,40.662652,-73.962081,3.011970e+09,...,2016-09-19 15:25:51,0.0,2501.0,12.2,63.2,18.4,2.4,1.6,35028.0,11.5
2,5,30191841,2019-06-06,2.831125,2.814634,2.857626,3.0,40.767326,-73.984310,1.010480e+09,...,2011-05-04 14:04:53,0.0,5195.0,72.9,3.6,8.4,0.0,12.3,141678.0,4.7
3,6,30191841,2022-01-04,2.831125,2.814634,2.870896,3.0,40.767326,-73.984310,1.010480e+09,...,2011-05-04 14:04:53,0.0,5195.0,72.9,3.6,8.4,0.0,12.3,141678.0,4.7
4,7,40356018,2019-11-08,2.868895,2.834724,2.855589,3.0,40.579920,-73.982090,3.069970e+09,...,2017-10-05 10:26:57,0.0,2185.0,38.0,2.2,40.8,0.0,19.1,20040.0,12.2


In [290]:
# now need to get these to the repeating CAMIS values
final_df = pd.merge(restaurant_data, df_insp, how="inner")
print(final_df.shape)
final_df.head()

(44786, 30)


,Unnamed: 0,CAMIS,GRADE DATE,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,Latitude,Longitude,BBL,...,INSPECTION_DATE,RESULT,Pop,White,Black,Hispanic,Native,Asian,Inc,Unemp
0,3,30112340,2019-03-04,2.801389,2.913240,2.792569,3.0,40.662652,-73.962081,3.011970e+09,...,2016-09-19 15:25:51,0.0,2501.0,12.2,63.2,18.4,2.4,1.6,35028.0,11.5
1,4,30112340,2021-08-05,2.801389,2.913240,2.634818,2.0,40.662652,-73.962081,3.011970e+09,...,2016-09-19 15:25:51,0.0,2501.0,12.2,63.2,18.4,2.4,1.6,35028.0,11.5
2,5,30191841,2019-06-06,2.831125,2.814634,2.857626,3.0,40.767326,-73.984310,1.010480e+09,...,2011-05-04 14:04:53,0.0,5195.0,72.9,3.6,8.4,0.0,12.3,141678.0,4.7
3,6,30191841,2022-01-04,2.831125,2.814634,2.870896,3.0,40.767326,-73.984310,1.010480e+09,...,2011-05-04 14:04:53,0.0,5195.0,72.9,3.6,8.4,0.0,12.3,141678.0,4.7
4,7,40356018,2019-11-08,2.868895,2.834724,2.855589,3.0,40.579920,-73.982090,3.069970e+09,...,2017-10-05 10:26:57,0.0,2185.0,38.0,2.2,40.8,0.0,19.1,20040.0,12.2


In [291]:
final_df.columns

Index(['Unnamed: 0', 'CAMIS', 'GRADE DATE', 'ZIPCODE', 'CUISINE DESCRIPTION',
       'VIOLATION CODE', 'GRADE', 'Latitude', 'Longitude', 'BBL',
       'ClosestWorkingTankDis', 'ClosestBadTankDis', 'BORO_Bronx',
       'BORO_Brooklyn', 'BORO_Manhattan', 'BORO_Queens', 'BORO_Staten Island',
       'CRITICAL FLAG_Not Critical', 'ClosestWorkingTankDate',
       'ClosestBadTankDate', 'INSPECTION_DATE', 'RESULT', 'Pop', 'White',
       'Black', 'Hispanic', 'Native', 'Asian', 'Inc', 'Unemp'],
      dtype='object')

In [292]:
final_df = df_insp[["CAMIS",
                     "ZIPCODE",
                     "CUISINE DESCRIPTION",
                     "GRADE",
                     "ClosestWorkingTankDis",
                     "ClosestBadTankDis",
                     "BORO_Bronx",
                     "BORO_Brooklyn",
                     "BORO_Manhattan",
                     "BORO_Queens",
                     "BORO_Staten Island",
                     "RESULT",
                     "Pop",
                     "White",
                     "Black",
                     "Hispanic",
                     "Native",
                     "Asian",
                     "Inc",
                     "Unemp",
                     ]]

final_df = final_df.dropna()
final_df["GRADE"].value_counts()

3.0    39922
2.0     3641
1.0     1223
Name: GRADE, dtype: int64

In [293]:
# maybe should get the number of times each restaurant has been inspected since inception... More frequently visited restaurants are probably higher risk
visit_counts = pd.DataFrame(final_df["CAMIS"].value_counts()).reset_index()
visit_counts = visit_counts.rename(columns={"index": "CAMIS", "CAMIS": "visit_count"})



In [294]:
final_df.to_csv("preprocessed_data/data_for_model.csv")